In [4]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import utils.cs_vqe as c
import itertools
import utils.qubit_conversion as q_conv

In [5]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import utils.cs_vqe as c
import itertools
import utils.qubit_conversion as q_conv

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (1.1324434021069383+0j) [] +
(-0.02581025441481776+0j) [X0 X1 Y2 Y3] +
(-0.012799934786435978+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839899145+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786435978+0j) [X0 X1 X3 X4] +
(-0.025159645839899145+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.0171145288217741+0j) [X0 X1 Y4 Y5] +
(-0.0009891049103608776+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.0009891049103608776+0j) [X0 X1 X5 X6] +
(-0.054698565746848046+0j) [X0 X1 Y6 Y7] +
(0.02581025441481776+0j) [X0 Y1 Y2 X3] +
(0.012799934786435978+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839899145+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786435978+0j) [X0 Y1 Y3 X4] +
(-0.025159645839899145+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.0171145288217741+0j) [X0 Y1 Y4 X5] +
(0.0009891049103608776+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.0009891049103608776+0j) [X0 Y1 Y5 X6] +
(0.054698565746848046+0j) [X0 Y1 Y6 X7] +
(0.002577905179419724+0j) [X0 Z1 X2] +
(0.004766148105680847+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923309548+0j) [X0 Z1 X2 X3 Z4 

UCCSD ansatz: 
 0.006478936130102658j [X0 X1 X2 Y3] +
0.006478936130102658j [X0 X1 Y2 X3] +
0.002056274402971196j [X0 X1 X4 Y5] +
0.002056274402971196j [X0 X1 Y4 X5] +
0.005083283191416127j [X0 X1 X6 Y7] +
0.005083283191416127j [X0 X1 Y6 X7] +
-0.006478936130102658j [X0 Y1 X2 X3] +
0.006478936130102658j [X0 Y1 Y2 Y3] +
-0.002056274402971196j [X0 Y1 X4 X5] +
0.002056274402971196j [X0 Y1 Y4 Y5] +
-0.005083283191416127j [X0 Y1 X6 X7] +
0.005083283191416127j [X0 Y1 Y6 Y7] +
-0.006360390047739343j [X0 Z1 Y2] +
-0.002645411873762702j [X0 Z1 Z2 Z3 Y4] +
0.0009221551709269771j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
-0.006478936130102658j [Y0 X1 X2 X3] +
0.006478936130102658j [Y0 X1 Y2 Y3] +
-0.002056274402971196j [Y0 X1 X4 X5] +
0.002056274402971196j [Y0 X1 Y4 Y5] +
-0.005083283191416127j [Y0 X1 X6 X7] +
0.005083283191416127j [Y0 X1 Y6 Y7] +
-0.006478936130102658j [Y0 Y1 X2 Y3] +
-0.006478936130102658j [Y0 Y1 Y2 X3] +
-0.002056274402971196j [Y0 Y1 X4 Y5] +
-0.002056274402971196j [Y0 Y1 Y4 X5] +
-0.005083283

In [6]:
ham = q_conv.QubitOperator_to_dict(ham_q, num_qubits)
anz_terms = list((q_conv.QubitOperator_to_dict(ucc_q, num_qubits)).keys())
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]
ham_noncon = {t:ham[t] for t in terms_noncon}
#ham_noncon
terms_context = list(ham.keys() - terms_noncon)
ham_context = {t:ham[t] for t in terms_context}
#ham_context
c.contextualQ_ham(ham_context)

True

In [7]:
from qiskit.aqua.algorithms import NumPyEigensolver

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
true_gs = np.real(result.eigenvalues)

print(true_gs)

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:38: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-terra')


[-3.16676548]


/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [8]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)
gs_noncon = c.find_gs_noncon(ham_noncon)
gs_noncon_energy = gs_noncon[0]
ep_state = gs_noncon[1]
ham_context = {p:c for p,c in ham.items() if p not in ham_noncon}
c.csvqe_approximations_heuristic(ham, ham_noncon, num_qubits, true_gs)
#c.contextualQ_ham(ham_context)

[array([-3.16676548]),
 [-3.14282474926818,
  -3.151872881862987,
  -3.1518728818629858,
  -3.1553177976071267,
  -3.159114851913753,
  -3.159114851913756,
  -3.1619490056505137,
  -3.1667654772544216,
  -3.166765477254425],
 [array([0.02394073]),
  array([0.0148926]),
  array([0.0148926]),
  array([0.01144768]),
  array([0.00765063]),
  array([0.00765063]),
  array([0.00481647]),
  array([-5.32907052e-15]),
  array([-8.8817842e-15])],
 [6, 4, 0, 2, 3, 1, 5, 7]]

In [9]:
A = {p:ep_state[1][index] for index, p in enumerate(model[1])}
A

{'YZZZZZYZ': 0.040172164101531696, 'IIIIIIZI': 0.9991927728078299}

# The epistemic state defines a classical probability distribution:

In [10]:
def ontic_prob(ep_state, ontic_state):
    
    if ep_state[0] != ontic_state[0]:
        return 0
    
    else:
        prod = 1
        for index, r in enumerate(ep_state[1]):
            f = 1/2 * abs(r + ontic_state[1][index])
            prod *= f
        
        return prod    

def epistemic_dist(ep_state):
    size_G = len(ep_state[0])
    size_Ci = len(ep_state[1])
    size_R = size_G + size_Ci
    
    ep_prob = {}
    
    ontic_states = list(itertools.product([1, -1], repeat=size_R))
    
    for o in ontic_states:
        o_state = [list(o[0:size_G]), list(o[size_G:size_R])]
        o_prob = ontic_prob(ep_state, o_state)
        
        if o_prob != 0:
            ep_prob[o] = o_prob
    
    return ep_prob

In [11]:
epistemic_dist(ep_state)

{(-1, -1, 1, -1, 1, 1, 1, 1, 1): 0.5198761682369156,
 (-1, -1, 1, -1, 1, 1, 1, 1, -1): 0.00020991381385028856,
 (-1, -1, 1, -1, 1, 1, 1, -1, 1): 0.47972021816699945,
 (-1, -1, 1, -1, 1, 1, 1, -1, -1): 0.00019369978223475102}

# Find a simultaneous eigenstate of the operators G_i and A:

In [12]:
from copy import deepcopy

G = model[0]
fixed_qubits = ['I' for i in range(num_qubits)]
basis_ops = []
mult_Z_indices = []

for index, g in enumerate(G):
    Z_indices = [i for i, p in enumerate(list(g)) if p == 'Z']
    if len(Z_indices) == 1:
        if ep_state[0][index] == -1:
            fixed_qubits[Z_indices[0]] = 1
        else:
            fixed_qubits[Z_indices[0]] = 0
    else:
        mult_Z_indices.append([ep_state[0][index], Z_indices])
        
print(fixed_qubits)

for Z_indices in mult_Z_indices:
    Z0, Z1 = Z_indices[1][0], Z_indices[1][1] 
    q0, q1 = fixed_qubits[Z0], fixed_qubits[Z1]
    
    if q0 != q1:
        if q0 != 'I':
            if Z_indices[0] == -1:
                fixed_qubits[Z1] = (q0+1)%2
            else:
                fixed_qubits[Z1] = q0
        else:
            if Z_indices[0] == -1:
                fixed_qubits[Z0] = (q1+1)%2
            else:
                fixed_qubits[Z0] = q1
        mult_Z_indices.remove(Z_indices)
            
for Z_indices in mult_Z_indices:
    if Z_indices[0] == -1:
        fixed = deepcopy(fixed_qubits)
        fixed[Z_indices[1][0]] = 0
        fixed[Z_indices[1][1]] = 1
        fixed = [str(i) for i in fixed]
        basis_ops.append(''.join(fixed))
        
        fixed = deepcopy(fixed_qubits)
        fixed[Z_indices[1][0]] = 1
        fixed[Z_indices[1][1]] = 0
        fixed = [str(i) for i in fixed]
        basis_ops.append(''.join(fixed))
    
print(basis_ops)

['I', 'I', 0, 1, 0, 0, 'I', 0]
['01010010', '11010000']


# Construct +1-Eigenstates of A(r)

In [13]:
r1 = ep_state[1][0]
r2 = ep_state[1][1]
#t = np.arctan((r2 + 1) / r1)
parity=3
q1=1
amp_ratio = (1 + r2 * (-1)**q1) / (r1 * (-1)**(1 + parity))
t = np.arctan(amp_ratio)

r1, r2, np.sin(t), np.cos(t)

(0.040172164101531696,
 0.9991927728078299,
 0.020090136786120604,
 0.9997981728348552)

In [14]:
from openfermion.linalg import LinearQubitOperator

ham_q = q_conv.dict_to_QubitOperator(ham, num_qubits)
ham_noncon_q = q_conv.dict_to_QubitOperator(ham_noncon, num_qubits)
ham_context_q = q_conv.dict_to_QubitOperator(ham_context, num_qubits)

#take expectation value algebraically

for p in np.linspace(0, 1, 1):
    psi = add_eigenstate(r1=r1, r2=r2, index=82, theta=0, n=0, num_qubits=num_qubits)
    expect = expectation(ham_q, psi)
    
    print('<H> w.r.t. simultaneous eigenstate:', expect)
    print('Noncontextual approximation:', gs_noncon_energy)
    print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match?

NameError: name 'add_eigenstate' is not defined

In [ ]:
from matplotlib import pyplot as plt

X=[]
Y=[]

for i in range(2**(num_qubits-1)):
    X.append(i)
    psi = add_eigenstate(r1=r1, r2=r2, index=i, theta=0, n=0, num_qubits=num_qubits)

    Y.append(expectation(ham_noncon_q, psi))

plt.plot(X, Y)
plt.hlines(gs_noncon_energy, 0, 127, color='r')
plt.show()

In [ ]:
import math

def random_vector(n):
    components = [np.random.normal() for i in range(n)]
    r = math.sqrt(sum(x*x for x in components))
    v = [x/r for x in components]
    return v

In [ ]:
import itertools
from itertools import chain, combinations
from copy import deepcopy

def bin_to_int(bits):
    bit_string = deepcopy(bits)
    if type(bit_string) == str:
        bit_string = [int(b) for b in bit_string]
    for index, b in enumerate(bit_string):
        bit_string[index] = b * 2 ** (len(bit_string)-index-1)
    return sum(bit_string)


def int_to_bin(integer, num_qubits):
    if integer >= 2**num_qubits:
        raise ValueError('Input integer larger than specified number of bits')
    bin_str=bin(integer)[2:]
    leading_0 = ''.join(['0' for i in range(num_qubits-len(bin_str))])
    return leading_0 + bin_str


def add_eigenstate(r1, r2, index, theta, n, num_qubits, custom_amp=None):
    """
    """
    B = list(itertools.product([0,1], repeat=num_qubits))
    b1 = list(B[index])
    b2 = deepcopy(b1)
    b2[0] = (b2[0]+1)%2
    b2[6] = (b2[6]+1)%2
    i1 = bin_to_int(b1)
    i2 = bin_to_int(b2)

    parity=sum(b1)
    q1 = b1[6]
    
    amp_ratio = (1 + r2 * (-1)**q1) / (r1 * (-1)**(1 + parity))
    t = ((-1)**n)*np.arctan(amp_ratio)
    psi = [0 for i in range(2**num_qubits)]

    if custom_amp is None:   
        psi[i1] = np.sin(t)*np.exp(1j*theta)
        psi[i2] = np.cos(t)*np.exp(1j*(theta - n*np.pi))
    else:
        psi[i1] = custom_amp[0]
        psi[i2] = custom_amp[1]
        
    return np.array(psi)


def expectation(op, state):
    state = np.array(state)
    conj_state = np.conjugate(state)
    O = LinearQubitOperator(op, num_qubits)
    
    O_state = O.matvec(state)
    expect = conj_state.dot(O_state)
    
    return expect


def discard_generator(ham_noncon, ham_context, generators):
    new_ham_noncon = deepcopy(ham_noncon)
    new_ham_context = deepcopy(ham_context)
    
    Z_indices = [g.index('Z') for g in generators]
    removed=[]
    
    for index in Z_indices:
        for p in ham_noncon:
            if p not in removed:
                if p[index] == 'Z':
                    new_ham_context[p] = ham_noncon[p]
                    del new_ham_noncon[p]
                    removed.append(p)
            
    return new_ham_noncon, new_ham_context


def rotate_operator(rotations, op):
    rot_op = {}
    
    for p in op.keys():
        p_ref = deepcopy(p)
        parity = 1
        coeff = op[p]
        for r in rotations:
            rotate_p = c.apply_rotation(r, p)
            p = list(rotate_p.keys())[0]
            parity *= rotate_p[p]
        
        rot_op[p] = parity * coeff
        
    return rot_op

def rotate_hamiltonian(rotations, ham, ham_noncon, ham_context):
    
    rot_ham={}
    rot_ham_noncon={}
    rot_ham_context={}

    for p in ham.keys():
        p_ref = deepcopy(p)
        parity = 1
        coeff = ham[p]
        for r in rotations:
            rotate_p = c.apply_rotation(r, p)
            p = list(rotate_p.keys())[0]
            parity *= rotate_p[p]
        
        rot_ham[p] = parity * coeff
        if p_ref in ham_noncon.keys():
            rot_ham_noncon[p] = parity * coeff
        else:
            rot_ham_context[p] = parity * coeff
            
    return rot_ham, rot_ham_noncon, rot_ham_context


def rotate_state(rotations, state):
    
    rot_state = deepcopy(state)
    
    for r in rotations:
        r_op = QubitOperator('', 1/np.sqrt(2)) - q_conv.dict_to_QubitOperator({r[1]: 1/np.sqrt(2)*1j}, num_qubits)
        r_op = LinearQubitOperator(r_op, num_qubits)
        rot_state = r_op.matvec(rot_state)
        
    return rot_state


def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


def find_eigenstate_indices(initial, removed_Z_indices):
    bins = []
    index_powerset = list(powerset(removed_Z_indices))
    
    for comb in index_powerset:
        initial_ref = list(deepcopy(initial))
        for c in comb:
            initial_ref[c] = str((int(initial_ref[c])+1)%2)
        bins.append(bin_to_int(''.join(initial_ref)))
    
    return bins


def expectation_optimiser(ham_n, ham_c, amps, initial_state, Z_indices, num_qubits, rotations=None):
    eigenstate_indices = find_eigenstate_indices(initial_state, Z_indices)
    
    psi = np.array([0 for i in range(2**num_qubits)], dtype=complex)
    for index, i in enumerate(eigenstate_indices):
        psi += (amps[index])*add_eigenstate(r1=r1, r2=r2, theta=0, n=0, index=i, num_qubits=num_qubits)
    
    if rotations is not None:
        psi = rotate_state(rotations, psi)
    
    expect_noncon = expectation(ham_n, psi)
    expect_context = expectation(ham_c, psi)
    
    return expect_noncon, expect_context

In [ ]:
generators = model[0]
#rot_G=[]
#rotations = (c.diagonalize_epistemic(model,fn_form,ep_state))[0]

#for g in G:
#    for r in rotations:
#        rotate_g = c.apply_rotation(r, g)
#        g = list(rotate_g.keys())[0]
#    rot_G.append(g)
    
#print(G, rot_G)
generators

In [ ]:
#take expectation value algebraically
from matplotlib import pyplot as plt
shots=10
initial_state = '01010010'

#rot_ham, rot_ham_noncon, rot_ham_context = rotate_hamiltonian(rotations, ham, ham_noncon, ham_context)

fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize = (21,12))
grid_pos = [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2)]
grid_pos.reverse()

X=list(range(1, shots+1))
print(X)
for index, grid in enumerate(grid_pos):
    
    removed_index = list(range(index+2, 7))
    removed_index.reverse()
    removed_generators = [generators[i] for i in removed_index]
    Z_indices = [g.find('Z') for g in removed_generators]
    print(removed_generators, Z_indices)
    
    new_ham_noncon, new_ham_context = discard_generator(ham_noncon, ham_context, removed_generators)

    #rot_ham_q = q_conv.dict_to_QubitOperator(ham, num_qubits)
    new_ham_noncon_q = q_conv.dict_to_QubitOperator(new_ham_noncon, num_qubits)
    new_ham_context_q = q_conv.dict_to_QubitOperator(new_ham_context, num_qubits)

    eigenstate_indices = find_eigenstate_indices(initial_state, Z_indices)
    amps=[random_vector(len(eigenstate_indices)) for i in range(shots)]
    Y_noncon=[]
    Y_context=[]
    Y_combined=[]
    Y_full=[]

    #A_op = QubitOperator('Y0 Z1 Z2 Z3 Z4 Z5 Y6 Z7', r1) + QubitOperator('Z6', r2)
    #A = LinearQubitOperator(A_op, num_qubits)

    for p in amps:
        expect_noncon, expect_context = expectation_optimiser(ham_n=new_ham_noncon_q, ham_c=new_ham_context_q, 
                                                              amps=p, initial_state=initial_state, Z_indices=Z_indices, num_qubits=num_qubits)

        #Y_full.append(expect_full)
        Y_noncon.append(expect_noncon)
        Y_context.append(expect_context)
        Y_combined.append(expect_noncon+expect_context)

        #print(p, expect_noncon + expect_context < gs_noncon_energy)
        #print((np.conjugate(psi)).dot(psi))
        #Verify +1-eigenstate
        #A_psi = A.matvec(psi)
        #print(psi_conj.dot(A_psi))
    
    axs[grid].plot(X, Y_noncon, color='orange')
    axs[grid].scatter(X, Y_context, color='blue', marker = 'x')
    axs[grid].plot(X, Y_context, color='blue', ls='--')
    axs[grid].scatter(X, Y_combined, color='black', marker = 'x')
    axs[grid].plot(X, Y_combined, color='black', ls='--')
    #plt.plot(range(shots), Y_combined, color='purple')
    axs[grid].hlines(gs_noncon_energy, 1, shots, color='r')
    axs[grid].hlines(true_gs, 1, shots, color='g')
    
    axs[grid].set_xticks(X)
    axs[grid].set_title("%i generators removed" % abs(index-5))
    if grid[0] == 1:
        axs[grid].set_xlabel('Sample Number',fontsize=16)
    if grid[1] == 0:
        axs[grid].set_ylabel('Energy (Ha)',fontsize=18)
    #print('<H> w.r.t. simultaneous eigenstate:', expect)
    #print('Noncontextual approximation:', gs_noncon_energy)
    #print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match? 

# Do we have any quantum corrections?

In [ ]:
G = model[0]
terms_context = list(ham_context.keys())
G

In [ ]:
new_ham_noncon, new_ham_context = discard_generator(ham_noncon, ham_context, ['ZIIIIIZI'])

q_corr_terms = []

for t in list(new_ham_context.keys()):
    commutes = []
    for g in G:
        if c.commute(t, g):
            commutes.append(g)
    #print(t, 'commutes with the noncontextual generators:', commutes)
    if commutes == G:
        q_corr_terms.append(t)
        
if q_corr_terms == []:
    print('No quantum correction')
else:
    print('Terms admitting quantum correction:', q_corr_terms)

In [ ]:
ucc = q_conv.QubitOperator_to_dict(ucc_q, num_qubits)
anz_terms = list(ucc.keys())
print(anz_terms)

In [ ]:
import cs_vqe_ansatz as c_anz

anz = c_anz.construct_ansatz(init_state = [1, 3, 6], paulis = anz_terms)
anz.draw()
#from qiskit import QuantumCircuit
#from qiskit.extensions import Initialize
#from qiskit.circuit import Parameter

#anz = QuantumCircuit(num_qubits) # We are redefining qc
#anz.initialize(psi)
#anz.ry(Parameter('x'), 3)
#anz.initialize(psi2)
#anz.rx(Parameter('y'), 3)

In [ ]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.opflow.primitive_ops import PauliOp
from qiskit import Aer

ham_qiskit = q_conv.dict_to_WeightedPauliOperator(ham)

seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
slsqp = SLSQP(maxiter=10000)

vqe = VQE(anz, optimizer=slsqp, quantum_instance=qi)
vqe_run    = vqe.compute_minimum_eigenvalue(operator=ham_qiskit)
vqe_result = vqe_run.optimal_value# + shift

print('VQE:', vqe_result, '|', 'Noncontextual Ground State:', gs_noncon_energy, 'True Ground State:', true_gs)

In [18]:
from qiskit.aqua.algorithms import NumPyEigensolver

order = [4, 0, 2, 3, 1, 5, 7, 6]
print(c.contextual_subspace_approximations(ham,model,fn_form,ep_state,order))

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
exact_energy = np.real(result.eigenvalues)

print(exact_energy)

[-3.14282474926818, -3.14282474926818, -3.146274540897046, -3.1473405764585216, -3.147340576458527, -3.1489133494755484, -3.1543011047508536, -3.154301104750834, -3.1667654772544176]
[-3.16676548]


In [ ]:
rotations = (c.diagonalize_epistemic(model,fn_form,ep_state))[0]
#rotations.reverse()
rotations

In [ ]:
rot_G=[]

for g in G:
    for r in rotations:
        rotate_g = c.apply_rotation(r, g)
        g = list(rotate_g.keys())[0]
    rot_G.append(g)
    
rot_G

In [ ]:
from openfermion.linalg import LinearQubitOperator

psi = add_eigenstate(r1=r1, r2=r2, theta=0, n=0, index=82, num_qubits=num_qubits)
rot_psi = rotate_state(rotations, psi)
rot_psi_conj = np.conjugate(rot_psi)

#rot_ham = c.get_reduced_hamiltonians(ham,model,fn_form,ep_state,order=[0,1,2,3,4,5,6,7])[8]

rot_ham, rot_ham_noncon, rot_ham_context = rotate_hamiltonian(rotations, ham, ham_noncon, ham_context)
rot_ham_noncon_q = q_conv.dict_to_QubitOperator(rot_ham, num_qubits)

#take expectation value algebraically
rot_H = LinearQubitOperator(rot_ham_noncon_q, num_qubits)
rot_H_psi = rot_H.matvec(rot_psi)
rot_expect = rot_psi_conj.dot(rot_H_psi)

print('<H> w.r.t. rotated simultaneous eigenstate:', rot_expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(rot_expect - gs_noncon_energy) < 10**-14) #sanity check - expectation values match? 

In [ ]:
#take expectation value algebraically
from matplotlib import pyplot as plt
shots=20
initial_state = '01010010'

removed_index = [2,3,0]
removed_generators = [rot_G[i] for i in removed_index]
Z_indices = [g.find('Z') for g in removed_generators]
print(removed_generators, Z_indices)

rot_ham, rot_ham_noncon, rot_ham_context = rotate_hamiltonian(rotations, ham, ham_noncon, ham_context)
new_rot_ham_noncon, new_rot_ham_context = discard_generator(rot_ham_noncon, rot_ham_context, removed_generators)
new_rot_ham_noncon_q = q_conv.dict_to_QubitOperator(new_rot_ham_noncon, num_qubits)
new_rot_ham_context_q = q_conv.dict_to_QubitOperator(new_rot_ham_context, num_qubits)

eigenstate_indices = find_eigenstate_indices(initial_state, Z_indices)
print(eigenstate_indices)

prob=0.999
amps=[]
for i in range(shots):
    amp_1 = [np.sqrt(prob)]
    amp_rest = [np.sqrt(1-prob)*a for a in random_vector(len(eigenstate_indices)-1)]
    amp_1 += amp_rest
    amps.append(amp_1)

X=list(range(1, shots+1))
Y_noncon=[]
Y_context=[]
Y_combined=[]
Y_full=[]

#A_op = QubitOperator('Y0 Z1 Z2 Z3 Z4 Z5 Y6 Z7', r1) + QubitOperator('Z6', r2)
#A = LinearQubitOperator(A_op, num_qubits)

for p in amps:
    expect_noncon, expect_context = expectation_optimiser(ham_n=new_rot_ham_noncon_q, ham_c=new_rot_ham_context_q, amps=p,
                                                          initial_state=initial_state,num_qubits=num_qubits,Z_indices=Z_indices,rotations=rotations)
    total = expect_noncon + expect_context
    #Y_full.append(expect_full)
    Y_noncon.append(expect_noncon)
    Y_context.append(expect_context)
    Y_combined.append(total)
    print(total, total < gs_noncon_energy, p)

#plt.plot(X, Y_noncon, color='orange')
#plt.scatter(X, Y_context, color='blue', marker = 'x')
#plt.plot(X, Y_context, color='blue', ls='--')
plt.scatter(X, Y_combined, color='black', marker = 'x')
plt.plot(X, Y_combined, color='black', ls='--')
plt.hlines(gs_noncon_energy, 1, shots, color='r')
#plt.hlines(true_gs, 1, shots, color='g')

In [ ]:
gs_noncon_energy

In [ ]:
(-3.14173689169366+0j) False [-0.6561365569846744, -0.006775889146723866, 0.029173298604223765, -0.7540476275164038], [82, 114, 66, 98]

In [ ]:
#take expectation value algebraically
from matplotlib import pyplot as plt
shots=100

H = LinearQubitOperator(ham_q, num_qubits)
indices = [82, 88, 98]
X=[random_vector(len(indices)) for i in range(shots)]
Y=[]
success=[]
#A_op = QubitOperator('Y0 Z1 Z2 Z3 Z4 Z5 Y6 Z7', r1) + QubitOperator('Z6', r2)
#A = LinearQubitOperator(A_op, num_qubits)

for amps in X:
    psi = np.array([0 for i in range(2**num_qubits)], dtype=complex)
    for index, i in enumerate(indices):
        psi += (amps[index])*add_eigenstate(r1=r1, r2=r2, theta=0, n=0, index=i, num_qubits=num_qubits)
    psi_conj = np.conjugate(psi)

    H_psi = H.matvec(psi)
    expect = psi_conj.dot(H_psi)
    print(expect<gs_noncon_energy, amps, expect, gs_noncon_energy)
    if expect<gs_noncon_energy:
        Y.append(expect)
        success.append(amps)
    #Verify +1-eigenstate
    #A_psi = A.matvec(psi)
    #print(psi_conj.dot(A_psi))

plt.plot(range(len(Y)), Y)
plt.scatter(range(len(Y)), Y)
plt.hlines(gs_noncon_energy, 0, shots, color='r')
#print('<H> w.r.t. simultaneous eigenstate:', expect)
#print('Noncontextual approximation:', gs_noncon_energy)
#print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match? 

In [ ]:
success_temp = success

In [ ]:
success_temp, [82, 88, 98]

In [ ]:
optimizers = [COBYLA(maxiter=80),SLSQP(maxiter=60)]
len(optimizers)

In [ ]:
def bin_to_int(bits):
    if type(bits) == str:
        bits = [int(b) for b in bits]
    for index, b in enumerate(bits):
        bits[index] = b * 2 ** (len(bits)-index-1)
    return sum(bits)

In [ ]:
from copy import deepcopy

v1 = [0,0,0,0,1,1,1,1]
v2 = deepcopy(v1)
v2[0] = (v2[0]+1)%2
v2[6] = (v2[6]+1)%2

In [ ]:
bin_to_int(v1), bin_to_int(v2)

In [ ]:
import itertools
from matplotlib import pyplot as plt

B = list(itertools.product([0,1], repeat=8))

X=[]
Y=[]

for b1 in B[0:128]:
    b1 = list(b1)
    b2 = deepcopy(b1)
    b2[0] = (b2[0]+1)%2
    b2[6] = (b2[6]+1)%2
    i1 = bin_to_int(b1)
    i2 = bin_to_int(b2)
    X.append(i1)
    Y.append(i2)
    print(i1, i2, i2-i1)
    
plt.scatter(X, Y, s=3)

In [ ]:
np.exp(0.6*1j)

In [ ]:
bin_to_int('01011010')

In [ ]:
from openfermion.linalg import get_sparse_operator, get_ground_state, generate_linear_qubit_operator

sparse_ham_q = get_ground_state(get_sparse_operator(ham_noncon_q, num_qubits).toarray())
sparse_ham_q

In [ ]:
add_eigenstate(r1=r1, r2=r2, index=19, theta=0, n=0, num_qubits=num_qubits)[145]

In [ ]:
discard_generator(ham_noncon, ham_context, ['IIIIIIIZ', 'IIIIIZII', 'IZIIIIII', 'IIZIIIII', 'IIIZIIII', 'IIIIZIII'])[0]

In [ ]:
eigenstate_indices = [82, 83, 86, 87, 90, 91, 94, 95]

In [ ]:
initial = 82
indices = []
removed_Z_indices = [7, 5, 4]
for i in removed_Z_indices:
    indices.append(initial)
    indices.append(initial+1)
    
    initial += 2**i

indices





In [ ]:
print(sorted(find_eigenstate_indices('01010010', [2, 3])))

In [ ]:
from openfermion.linalg import get_sparse_operator, get_ground_state

gs = get_ground_state(get_sparse_operator(ham_q, num_qubits).toarray())
print(gs[0])

In [ ]:
amp_list = [abs(a)**2 for a in list(gs[1])]
sig_amp_list = [(str(index), a) for index, a in enumerate(amp_list) if a > 0.00001]
XY = list(zip(*sig_amp_list))
X = XY[0]
Y = XY[1]

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(12, 6), dpi=80)

plt.bar(X, Y)
plt.xticks(rotation=90)
plt.show()

In [ ]:
gs

In [ ]:
int_to_bin(991, 10)

In [ ]:
B = list(itertools.product([0,1], repeat=num_qubits))
for index in range(128):
    b1 = list(B[index])
    b2 = deepcopy(b1)
    b2[0] = (b2[0]+1)%2
    b2[6] = (b2[6]+1)%2
    i1 = bin_to_int(b1)
    i2 = bin_to_int(b2)
    print(i1, i2)

In [ ]:
eigenstate_indices = find_eigenstate_indices('01010010', [2, 3])
eigenstates = [int_to_bin(i, num_qubits) for i in eigenstate_indices]
eigenstate_strings = []
for s in eigenstates:
    eigenstate_strings.append(s)
    b1 = list(s)
    b2 = deepcopy(b1)
    b2[0] = str((int(b2[0])+1)%2)
    b2[6] = str((int(b2[6])+1)%2)
    eigenstate_strings.append(''.join(b2))

eigenstate_strings

In [ ]:
eigenstate_strings_indices = []
for s in eigenstate_strings:
    X_index = ['I' for i in range(num_qubits)]
    for index, bit in enumerate(list(s)):
        if bit == '1':
            X_index[index] = 'X'
    eigenstate_strings_indices.append(''.join(X_index))
eigenstate_strings_indices

In [ ]:
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
%matplotlib inline
param_chars = ['α','β','γ','δ','ε','ζ','η','θ','ι','κ','λ','μ','ν','ξ','ο','π','ρ','ς','σ','τ','υ','φ','χ','ψ','ω']

indices = find_eigenstate_indices('01010010', [2, 3, 0])
print(indices)
psi = np.array([0 for i in range(2**num_qubits)], dtype=complex)
for index, i in enumerate(indices):
    psi += (np.sqrt(1/len(indices)))*add_eigenstate(r1=r1, r2=r2, theta=0, n=0, index=i, num_qubits=num_qubits)

anz = QuantumCircuit(8)
anz.initialize(psi)
for q in range(8):
    anz.ry(Parameter(param_chars[q]), q)
#anz.measure_all()

anz.draw()
#backend = BasicAer.get_backend('qasm_simulator')
#job = execute(anz, backend)
#plot_histogram(job.result().get_counts(), color='midnightblue', title="New Histogram")

In [ ]:
bin_to_int('11110000')

In [20]:
print(clear([1, 2, 3]))

NameError: name 'clear' is not defined

In [ ]:
from qiskit.circuit.parameter import Parameter

r1 = ep_state[1][0]
r2 = ep_state[1][1]
#t = np.arctan((r2 + 1) / r1)
parity=3
q1=1
amp_ratio = (1 + r2 * (-1)**q1) / (r1 * (-1)**(1 + parity))
t = np.arctan(amp_ratio)

qc = QuantumCircuit(8)
qc.x(6), qc.x(4)
qc.ry(Parameter('y'), 7)
qc.cx(7, 1)
qc.x(7)

qc.rx(Parameter('x'), 5)
qc.cx(5, 4)
#qc.cx(7, 1)

#qc = QuantumCircuit(8)
#qc.ry(2*t, 7)
#qc.x(6), qc.x(4)
#qc.cx(7, 1)
#qc.x(7)
#qc.z(1)
#qc.rz(Parameter('x'),0)

#qc.measure_all()
print(qc.draw())
#backend = BasicAer.get_backend('qasm_simulator')
#job = execute(qc, backend)
#plot_histogram(job.result().get_counts(), color='midnightblue', title="New Histogram")

In [ ]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.opflow.primitive_ops import PauliOp
from qiskit import Aer

ham_qiskit = q_conv.dict_to_WeightedPauliOperator(ham)

seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
slsqp = SLSQP(maxiter=10000)

vqe = VQE(qc, optimizer=slsqp, quantum_instance=qi)
vqe_run    = vqe.compute_minimum_eigenvalue(operator=ham_qiskit)
vqe_result = vqe_run.optimal_value# + shift

print('VQE:', vqe_result, '|', 'Noncontextual Ground State:', gs_noncon_energy, 'True Ground State:', true_gs)

In [ ]:
int_to_bin(98, 8)

In [ ]:
from openfermion.linalg import get_sparse_operator, get_ground_state
from matplotlib import pyplot as plt

gs = get_ground_state(get_sparse_operator(ham_q, num_qubits).toarray())

amp_list = [abs(a)**2 for a in list(gs[1])]
sig_amp_list = sorted([(str(index), a) for index, a in enumerate(amp_list) if a > 10**-30], key=lambda x:x[1])
sig_amp_list.reverse()

XY = list(zip(*sig_amp_list))
X = XY[0]
Y = XY[1]
Y_log = [np.log10(a) for a in Y]

from matplotlib.pyplot import figure

figure(figsize=(15, 10), dpi=200)

plt.grid(zorder=0)
plt.bar(X, Y, zorder=2, label='Probability of observing basis state')
plt.bar(X, Y_log, zorder=3, label = 'log (base 10) of probability')
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
qubit_map={}
B = list(itertools.product([0,1], repeat=num_qubits))
for index in range(2**(num_qubits)):
    b1 = list(B[index])
    b2 = deepcopy(b1)
    b2[0] = (b2[0]+1)%2
    b2[6] = (b2[6]+1)%2
    i1 = bin_to_int(b1)
    i2 = bin_to_int(b2)
    qubit_map[i1] = [(i1, b1), (i2, b2)]

for i in find_eigenstate_indices('01010010', [2,3,0]):
    print(qubit_map[i])